## 뽐뿌 특가 게시물 크롤링
- https://www.ppomppu.co.kr/robots.txt 확인 결과 zboard는 Allow이며, 그중 뽐뿌 게시판은 disallow가 없으므로 맘편히 크롤링함
- 인기, 핫 게시물은 1달이 지나면 뽐뿌 게시판에 마크가 사라짐
    - 인기 게시판에서 인기/핫 여부를 크롤링하여 두 데이터를 merge 해주어야함
- 크롤링 기준 2019년 1월 1일 ~ 2023년 6월 29일 약 3년 6개월 데이터

In [75]:
# 패키지 import
import requests
import pandas as pd
import bs4
import time

from datetime import datetime
from bs4 import BeautifulSoup
from tqdm import notebook, tqdm_notebook

import tqdm

In [76]:
def get_datas(items):
    """ 뽐뿌 게시판 크롤링 함수
    뽐뿌 사이트 내 뽐뿌 게시판에 작성된 특가 게시글 크롤링
    Args:
        items - BS4를 사용한 HTML 파서 데이터. 뽐뿌 게시판 1페이지의 게시글 정보가 담긴 데이터
    Returns:
        DataFrame : 아래의 컬럼 정보를 가진 DataFrame을 Return
            - item_no : 게시글 번호
            - writet : 작성자
            - title : 게시글 제목
            - end : 게시글 종료 여부
            - comment : 게시글에 달린 댓글 수
            - date : 게시글 등록일
            - recommend : 추천수
            - opposite : 반대수
            - view : 조회수
            - category : 게시글의 카테고리 e.g) 디지털
            - ULR : 게시글 접속 URL
    """
    data = {
        "item_no": [],
        "writer": [],
        "title": [],
        "end": [],
        "comment": [],
        "date": [],
        "recommend": [],
        "opposite": [],
        "view": [],
        "category": [],
        "URL": [],
    }

    for item in items:
        item_no = item.find("td", "eng list_vspace").text.strip()
        data["item_no"].append(item_no)

        writer = item.find("span", "list_name")
        data["writer"].append(writer.text if writer else "Image_name")

        title = item.find("font")
        data["title"].append(title.text if title else "해당글은 게시중단요청에 의해 블라인드 처리된 글입니다.")

        data["end"].append(bool(item.find("img", {"src": "/zboard/skin/DQ_Revolution_BBS_New1/end_icon.PNG"})))

        comment = item.find("span", "list_comment2")
        data["comment"].append(comment.text.strip() if comment else 0)

        data["date"].append(item.find_all("td", "eng list_vspace")[1]["title"])

        rec_opp = item.find_all("td", "eng list_vspace")[2].text.split('-')
        data["recommend"].append(rec_opp[0] if len(rec_opp) > 1 else 0)
        data["opposite"].append(rec_opp[1] if len(rec_opp) > 1 else 0)

        data["view"].append(item.find_all("td", "eng list_vspace")[3].text)

        cat = item.find("span", {"style": "color:#999;font-size:11px;"})
        data["category"].append(cat.text if cat else "")

        data["URL"].append(f"https://www.ppomppu.co.kr/zboard/view.php?id=ppomppu&no={item_no}")

    return pd.DataFrame(data)

In [78]:
def get_pop_post(items):
    """ 인기 게시판 크롤링 함수
    뽐뿌 사이트 내 뽐뿌 게시판에 작성된 특가 게시글 중 인기/Hot 게시물 크롤링
    Args:
        items - BS4를 사용한 HTML 파서 데이터. 뽐뿌 핫 게시판 1페이지의 게시글 정보가 담긴 데이터
    Returns:
        DataFrame : 아래의 컬럼 정보를 가진 DataFrame을 Return
            - item_no : 게시글 번호
            - pop : 인기 게시글 여부
            - hot : 핫 게시글 여부
    """
    data = {
        "item_no": [],
        "pop": [],
        "hot": [],
    }

    for item in items:
        item_no = item.find("td", "eng list_vspace").text.strip()
        data["item_no"].append(item_no)


        data["pop"].append(bool(item.find("img", {"src": "/images/menu/pop_icon2.jpg"})))
        data["hot"].append(bool(item.find("img", {"src": "/images/menu/hot_icon2.jpg"})))

    return pd.DataFrame(data)

In [77]:
# 특가 게시물 크롤링
# 파라미터
end_page = 5900
base_url = "https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page={}"
temp_list = []

# 시작 시간 체크
start_time = time.time()

# Crawling
for page in tqdm.tqdm(range(2, end_page)):
    response = requests.get(base_url.format(page))
    
    if response.status_code == 200:
        html = BeautifulSoup(response.text, 'html.parser')
        items = html.find_all("tr", ["common-list0", "common-list1"])
        temp_df = get_datas(items)
        temp_list.append(temp_df)
        time.sleep(0.1)

# 최종 데이터 프레임 저장
df = pd.concat(temp_list).reset_index(drop=True)

# 소요시간 체크
elapsed_time = time.time() - start_time
print(f"Crawling took {round(elapsed_time, 2)} seconds")

In [79]:
# 인기/핫 게시물 크롤링
# 파라미터
end_page = 1600
hot_url = "https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&page={}&hotlist_flag=999"
temp_list = []

# 시작 시간 체크
start_time = time.time()

# Crawling
for page in tqdm.tqdm(range(2, end_page)):
    response = requests.get(hot_url.format(page))
    
    if response.status_code == 200:
        html = BeautifulSoup(response.text, 'html.parser')
        items = html.find_all("tr", ["common-list0", "common-list1"])
        temp_df = get_pop_post(items)
        temp_list.append(temp_df)
        # time.sleep(0.1)

# 최종 데이터 프레임 저장
hot_df = pd.concat(temp_list).reset_index(drop=True)

# 소요시간 체크
elapsed_time = time.time() - start_time
print(f"Crawling took {round(elapsed_time, 2)} seconds")

100%|████████████████████████████████████████████████████████████████████████████| 1598/1598 [17:35<00:00,  1.51it/s]

Crawling took 1055.45 seconds


In [ ]:
df = df.merge(hot_df, how="left").fillna(False)

In [100]:
# 데이터 csv 저장
now = str(datetime.now())
df.to_csv(f"./datas/{now}_{len(df)}개.csv", index=False)